# ETL Pipeline

## Extract Data

In [1]:
# %pip install -q tweepy
# %pip install -q python-decouple

In [2]:
# import necessay packages
import pandas as pd
import tweepy as tw
# from decouple import config

In [3]:
# config
consumer_key = 'ALS2SK37YuglyLYSyGGpfrbmY'
consumer_secret = '2Vyr3pBaHDsNaVX9IslmRHq25xR4zJjQjHpjUFTZzXj74rkvwS'
access_token = '1521183878576230400-4DzxTK7h5tI1gQ1f4QwoEWqfn0FNuV'
access_token_secret = 'sft4cbN41jMtVneJLD794rYRGJraHbb9ug4YELbZHFSyV'
bearer_token = 'AAAAAAAAAAAAAAAAAAAAAA6ScAEAAAAAnv1iKJAyg2INl1nA7%2BB0z%2FJbpK4%3DbrVth6B429DvCW0vsS4ufO6pLzmsAnhyYpaPRfWGoMWBKc6uRM'
user_id = '90676816'

In [4]:
# connect API
permission = tw.OAuthHandler(consumer_key, consumer_secret) # this object of ours will load the authorization request
permission.set_access_token(access_token, access_token_secret) # get ownership of our access object
client = tw.Client(bearer_token=bearer_token)
api = tw.API(permission) # use the API class, native to the library to generate the connection

In [5]:
query = 'Atletico-MG -filter:retweets OR Atlético Mineiro -filter:retweets OR #Galo -filter:retweets OR #ClubeAtléticoMineiro -filter:retweets'
galo_tweets = api.search_tweets(q=query, tweet_mode='extended', count=100)

id = []
created_at = []
full_text = []
favorite_count = []
retweet_count = []

for twitters in galo_tweets:
    id.append(twitters.id)
    created_at.append(twitters.created_at)
    full_text.append(twitters.full_text)
    favorite_count.append(twitters.favorite_count)
    retweet_count.append(twitters.retweet_count)

    # create dataframe
    df_final = {
      'tweet_id': id, 'created_at':created_at, 'text':full_text, 'likes':favorite_count, 
      'retweets':retweet_count
    }

In [6]:
search_tweets_df = pd.DataFrame(df_final)
search_tweets_df

,tweet_id,created_at,text,likes,retweets
0,1617705848025018376,2023-01-24 02:08:24+00:00,Primeira rodada do Campeonato Mineiro 2023:\n\...,5,0
1,1617693343345147904,2023-01-24 01:18:42+00:00,@zegustavo97 Será que ele não lembrou um certo...,1,0
2,1617670116870868993,2023-01-23 23:46:25+00:00,O primeiro jogo do Galo no Mineirão será contr...,482,13
3,1617665620631498752,2023-01-23 23:28:33+00:00,"Hulk, do Atlético-MG, estabelece meta de gols ...",2,1
4,1617654672508215297,2023-01-23 22:45:03+00:00,"Hulk, do Atlético-MG, estabelece meta de gols ...",1,0
...,...,...,...,...,...
95,1616856355368828929,2023-01-21 17:52:49+00:00,Atlético-MG e Cruzeiro vão estrear no Campeona...,0,0
96,1616846678371303431,2023-01-21 17:14:22+00:00,Estaduais:\nMineiro. Jogos deste sábado. • 15h...,1,0
97,1616837664778645504,2023-01-21 16:38:33+00:00,Atlético-MG x Caldense: Galo inicia a busca pe...,42,1
98,1616835539675619331,2023-01-21 16:30:06+00:00,COMEÇANDO DAQUI A 3 HORAS!\n\n16:30\nCampeonat...,0,0


In [7]:
# empty lists are created that will store the data extracted from the API call
id = []
date = []
text = []
retweets = []
answers = []
likes = []
mentions = []

# API call script with loop and pagination
for tweet in tw.Paginator(
    client.get_users_tweets, id=user_id, exclude=['retweets', 'replies'], #start_time = '2023-01-01T00:00:00Z',
    tweet_fields=['id', 'text', 'created_at', 'public_metrics'], max_results=100).flatten(limit=3200):

    # inserting the results in the lists
    id.append(tweet['id'])
    date.append(tweet['created_at'])
    text.append(tweet['text'])
    retweets.append(tweet['public_metrics']['retweet_count'])
    answers.append(tweet['public_metrics']['reply_count'])
    likes.append(tweet['public_metrics']['like_count'])
    mentions.append(tweet['public_metrics']['quote_count'])

    # create dataframe
    results = {
      'tweet_id':id, 'date':date, 'text':text, 'likes':likes, 
      'retweets':retweets, 'answers':answers, 'mentions':mentions
    }

In [8]:
paginator_df = pd.DataFrame(results)
paginator_df

,tweet_id,date,text,likes,retweets,answers,mentions
0,1617903320944844800,2023-01-24 15:13:05+00:00,🎥⚽ Os gols de Vargas (2) e Calebe na vitória p...,938,66,48,19
1,1617871212687855616,2023-01-24 13:05:30+00:00,"🤔 Desculpa, qual foi a pergunta? 🤣\n\n🎙️ Se li...",627,35,10,8
2,1617674919000539137,2023-01-24 00:05:30+00:00,🤬 Não deixe o Otávio no vácuo! 🤭\n\n🎥 O Desafi...,592,33,4,2
3,1617637170268758037,2023-01-23 21:35:30+00:00,🖤🤍 t_amo_família_do_Hulk 🖤🤍 https://t.co/cTfGl...,3305,199,24,13
4,1617621944995569684,2023-01-23 20:35:00+00:00,"💪🏽 No fim de semana, o Atlético estreou com vi...",822,42,1,1
...,...,...,...,...,...,...,...
819,1598083199061479424,2022-11-30 22:35:00+00:00,"🇦🇷🐔 Chacho já começou os trabalhos por aqui, h...",1862,164,17,10
820,1598068099546316803,2022-11-30 21:35:00+00:00,⚽ Nome: Cláudio André Mergen Taffarel\n⭐ Apeli...,1448,94,14,4
821,1598059294586306560,2022-11-30 21:00:00+00:00,Vamo que vamo que essa semana tem jogo na sext...,307,24,2,1
822,1598025666460459008,2022-11-30 18:46:23+00:00,"🎙️ Camisa 10, argentino, canhoto... ⭐🇦🇷\n\n#FI...",1786,141,28,25


In [9]:
# test = pd.read_parquet('atletico_own_tweets.parquet')
# test